In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from sklearn.metrics import classification_report


import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s

from pprint import pprint

In [2]:
a00, a01 = h.import_desired_data("A", "15T")

w3 = w.Window_Detect2(a01)
w3.run_analyis_and_guess(w.make_ewm_smooth, )
# w3.analyze_window_change(w.make_ewm_smooth)

In [3]:
w3.plot_guesses(make_guesses=True, timedelta=15, z=1)

In [4]:
w3.plot_guesses(make_guesses=True, timedelta=15, z=2)

In [5]:
metrics = {}

for z in [1, 2]:
    truth = w3.window

    w3.run_guess(z=z)
    # change time interpolation to have a numerical index instead od a datetime one 
    reset_interp = w3.interp_guess.reset_index()
    choices = reset_interp[0]

    s1 = s.Scores(exp=a01, choices=choices, guess_times_ix=w3.guess_times.index)
    metrics[f"z={z}"] = {}
    metrics[f"z={z}"]["standard"] = s1.calc_standard_metrics()
    metrics[f"z={z}"]["drdr"] = s1.calc_drdr_metrics()
    metrics[f"z={z}"]["custom"] = {}
    metrics[f"z={z}"]["custom"]["pre_interp"] = s1.calc_hit_scores() 
    metrics[f"z={z}"]["custom"]["post_interp"] = s1.calc_hit_scores(guess_times_ix = s.identify_changed_ix(choices) ) 

    


In [9]:
short_metrics = {}
for k, v in metrics.items():
    short_metrics[k] = {}

    short_metrics[k]["standard"] = {}

    short_metrics[k]["standard"]["macro avg f1-score"] = np.round(v["standard"]["macro avg"]["f1-score"],3)



    short_metrics[k]["custom"] = {}

    short_metrics[k]["custom"]["(hits + near hits)/guesses"] = v["custom"]["post_interp"]["(hits + near hits)/guesses"]

    short_metrics[k]["custom"]["guesses/actions"] = v["custom"]["post_interp"]["guesses/actions"]



    short_metrics[k]["drdr"] = {}

    short_metrics[k]["drdr"]["unbounded acc"] = v["drdr"]["unbounded acc"]

    short_metrics[k]["drdr"]["opening acc"] = v["drdr"]["bounded acc"]


In [10]:
pprint(short_metrics, width=20, sort_dicts=False)

{'z=1': {'standard': {'macro avg f1-score': 0.724},
         'custom': {'(hits + near hits)/guesses': 1.0,
                    'guesses/actions': 0.692},
         'drdr': {'unbounded acc': -2.19,
                  'opening acc': 0.5}},
 'z=2': {'standard': {'macro avg f1-score': 0.582},
         'custom': {'(hits + near hits)/guesses': 0.636,
                    'guesses/actions': 0.846},
         'drdr': {'unbounded acc': -0.19,
                  'opening acc': 0.602}}}


In [13]:
# with open("../constants/ewm_metrics_230629.json", 'w') as convert_file:
#      convert_file.write(json.dumps(metrics, default=str))


In [7]:
pprint(metrics, width=20, sort_dicts=False)

{'z=1': {'standard': {'0.0': {'precision': 0.6608187134502924,
                              'recall': 0.710691823899371,
                              'f1-score': 0.6848484848484848,
                              'support': 159},
                      '1.0': {'precision': 0.7850467289719626,
                              'recall': 0.7433628318584071,
                              'f1-score': 0.7636363636363637,
                              'support': 226},
                      'accuracy': 0.7298701298701299,
                      'macro avg': {'precision': 0.7229327212111275,
                                    'recall': 0.727027327878889,
                                    'f1-score': 0.7242424242424242,
                                    'support': 385},
                      'weighted avg': {'precision': 0.7337421719123638,
                                       'recall': 0.7298701298701299,
                                       'f1-score': 0.7310979929161748,
                